In [1]:
import geopandas as gpd
import geoengine as ge

/home/droenner/git/geoengine-python/env/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
ge.initialize("http://localhost:3030/api", credentials=("admin@localhost", "admin"))

In [3]:
session = ge.get_session()
user_id = session.user_id
session

Server:              http://localhost:3030/api
User Id:             d5328854-6190-4af9-ad69-4e74b0961ac9
Session Id:          f0d791ce-7823-43c8-a307-69316ebcab88
Session valid until: 2023-03-27T18:49:11.517Z

In [4]:
used_quota_start = ge.get_quota(user_id)['used']
used_quota_start

0

In [5]:
bounds_array = [421398.30688394164, 5681079.293017321, 476200.5934965983, 5727832.747780239]
xmin = bounds_array[0]
ymin = bounds_array[1]
xmax = bounds_array[2]
ymax = bounds_array[3]

(xmin, ymin, xmax, ymax)

(421398.30688394164, 5681079.293017321, 476200.5934965983, 5727832.747780239)

In [6]:
sentinel2_band_scl_workflow = ge.unstable.workflow_blueprints.sentinel2_band("SCL")
sentinel2_band_scl_workflow.to_workflow_dict()

{'type': 'Raster',
 'operator': {'type': 'GdalSource',
  'params': {'data': {'type': 'external',
    'providerId': '5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5',
    'layerId': 'UTM32N:SCL'}}}}

In [7]:
senttinel2_band_scl_workflow_id = ge.register_workflow(sentinel2_band_scl_workflow.to_workflow_dict())
senttinel2_band_scl_workflow_id

11847314-a330-5360-b27b-50cf9eca6613

In [8]:
from datetime import datetime
time_start = datetime(2021, 1, 1)
time_end = datetime(2021, 12, 31)

study_area = ge.api.RasterQueryRectangle(
    spatialBounds=ge.SpatialPartition2D(xmin, ymin, xmax, ymax).to_api_dict(),
    timeInterval=ge.TimeInterval(time_start, time_end).to_api_dict(),
    spatialResolution=ge.SpatialResolution(10.0, 10.0).to_api_dict(),
)
study_area

{'spatialBounds': {'upperLeftCoordinate': {'x': 421398.30688394164,
   'y': 5727832.747780239},
  'lowerRightCoordinate': {'x': 476200.5934965983, 'y': 5681079.293017321}},
 'timeInterval': {'start': '2021-01-01T00:00:00.000+00:00',
  'end': '2021-12-31T00:00:00.000+00:00'},
 'spatialResolution': {'x': 10.0, 'y': 10.0}}

In [9]:
status_download = {}

for b in ["B02", "B03", "B04", "B08", "SCL"]:
    print(b)
    sentinel2_band_workflow = ge.unstable.workflow_blueprints.sentinel2_band(b)
    sentinel2_band_workflow_id = ge.register_workflow(sentinel2_band_workflow.to_workflow_dict())
    sentinel2_band_workflow_dataset_task = sentinel2_band_workflow_id.save_as_dataset(study_area, f"sentinel2_nrw_crop_10m_{b}")
    sentinel2_band_workflow_dataset_task.wait_for_finish(print_status=False)
    print(sentinel2_band_workflow_dataset_task.get_status())
    status_download[b] = sentinel2_band_workflow_dataset_task.get_status()

status_download

B02
status=completed, time_started=2023-03-27 17:49:12.057000+00:00, info={'dataset': '1f171564-ddb4-4538-8737-1becf6b2d78b', 'upload': 'f0a17c38-aa49-4803-98a6-0d0f01a31795'}, time_total=00:52:08
B03
status=completed, time_started=2023-03-27 18:41:21.940000+00:00, info={'dataset': 'f387dce2-c081-4652-aa2a-57216df6c9a2', 'upload': '8d2a0356-22b3-493a-81fb-ec08a4a2b715'}, time_total=00:53:21
B04
status=completed, time_started=2023-03-27 19:34:46.857000+00:00, info={'dataset': 'e414ab68-6644-44f7-b81c-05d0e3efdbfb', 'upload': '13723a8c-f0b1-4430-a1ce-fc86e2c1f81e'}, time_total=00:54:57
B08
status=completed, time_started=2023-03-27 20:29:46.936000+00:00, info={'dataset': '1b351f15-e55c-4747-8906-8444e46c1711', 'upload': '044ded38-32d1-4f70-962e-af8ef0e4f9bd'}, time_total=00:56:15
SCL
status=completed, time_started=2023-03-27 21:26:07.169000+00:00, info={'dataset': 'f85fbcad-0df0-40bc-84ec-b8e8358cbffb', 'upload': '2a91cfe2-1e27-484e-bff1-cfd2f0b98ece'}, time_total=00:12:29


{'B02': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 17, 49, 12, 57000, tzinfo=datetime.timezone.utc),info = {'dataset': '1f171564-ddb4-4538-8737-1becf6b2d78b', 'upload': 'f0a17c38-aa49-4803-98a6-0d0f01a31795'}, time_total = '00:52:08'),
 'B03': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 18, 41, 21, 940000, tzinfo=datetime.timezone.utc),info = {'dataset': 'f387dce2-c081-4652-aa2a-57216df6c9a2', 'upload': '8d2a0356-22b3-493a-81fb-ec08a4a2b715'}, time_total = '00:53:21'),
 'B04': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 19, 34, 46, 857000, tzinfo=datetime.timezone.utc),info = {'dataset': 'e414ab68-6644-44f7-b81c-05d0e3efdbfb', 'upload': '13723a8c-f0b1-4430-a1ce-fc86e2c1f81e'}, time_total = '00:54:57'),
 'B08': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 20, 29, 46, 936000, tzinfo=datetime.timezone.utc),info = {'dataset': '1b351f15-e55c-4747-8906-8

In [10]:
used_quota_download = ge.get_quota(user_id)['used'] - used_quota_start
used_quota_download

17460

Local Sentinel2 to month aggregates workflow


In [11]:
## IF you ran the workflow, you can get the ids from the status
band_data_map = { name: task_status.info['dataset'] for name, task_status in status_download.items() }
## IF you have the data already added, youn simply define the ids.
#band_data_map = {
#    'b02': '48d30acd-a378-4f2a-89b3-d73430c0f29e',
#    'b03': '200ed06c-fcd1-41a3-b2dd-68b5d0b338a9',
#    'b04': 'b9ececef-5a0c-4294-be16-334e8017f60f',
#    'b08': '8e635367-35ff-42f9-bb21-88ae9db8be8d',
#    'scl': '64636605-10da-4576-bf21-a265cc1f7d9c'
#    }

band_data_id_map = {name.lower(): 
    ge.api.InternalDataId(
        type="internal",
        datasetId=id) for name, id in band_data_map.items()
} 

band_data_id_map

{'b02': {'type': 'internal',
  'datasetId': '1f171564-ddb4-4538-8737-1becf6b2d78b'},
 'b03': {'type': 'internal',
  'datasetId': 'f387dce2-c081-4652-aa2a-57216df6c9a2'},
 'b04': {'type': 'internal',
  'datasetId': 'e414ab68-6644-44f7-b81c-05d0e3efdbfb'},
 'b08': {'type': 'internal',
  'datasetId': '1b351f15-e55c-4747-8906-8444e46c1711'},
 'scl': {'type': 'internal',
  'datasetId': 'f85fbcad-0df0-40bc-84ec-b8e8358cbffb'}}

In [12]:
status_agg = {}

for b in ["B02", "B03", "B04", "B08", "NDVI"]:
    print(b)
    sentinel2_band_workflow = ge.unstable.workflow_blueprints.s2_cloud_free_monthly_band_custom_input(b, band_data_id_map, granularity="months", window_size=1, aggregation_type="mean")
    sentinel2_band_workflow_id = ge.register_workflow(sentinel2_band_workflow.to_workflow_dict())
    sentinel2_band_workflow_dataset_task = sentinel2_band_workflow_id.save_as_dataset(study_area, f"sentinel2_nrw_crop_10m_cf_monthly_{b}")
    sentinel2_band_workflow_dataset_task.wait_for_finish(print_status=False)
    print(sentinel2_band_workflow_dataset_task.get_status())
    status_agg[b] = sentinel2_band_workflow_dataset_task.get_status()

status_agg

B02
status=completed, time_started=2023-03-27 21:38:38.532000+00:00, info={'dataset': 'c533b58b-9c09-4eba-9fcb-cc468aa0afc9', 'upload': '10c89ab5-d2f4-4693-8280-0b0c9eaa2aca'}, time_total=00:13:18
B03
status=completed, time_started=2023-03-27 21:51:59.610000+00:00, info={'dataset': 'c33e80a6-50d6-411c-a8b7-9397e0190e6d', 'upload': '5b52968a-ed72-400f-9c40-5d3210a206bc'}, time_total=00:13:10
B04
status=completed, time_started=2023-03-27 22:05:10.714000+00:00, info={'dataset': 'd917ff6f-5d1d-44da-a543-91509a2b7c3c', 'upload': 'eff2fa92-aac2-4d3e-bdfd-4d4703132a49'}, time_total=00:13:20
B08
status=completed, time_started=2023-03-27 22:18:31.762000+00:00, info={'dataset': '0970f5a8-66cc-4348-91af-1ec413edb6ce', 'upload': '229ff1ae-4afe-48c1-8aee-223a57e4e9e8'}, time_total=00:13:13
NDVI
status=completed, time_started=2023-03-27 22:31:47.842000+00:00, info={'dataset': 'd62c540f-1ad5-450c-b81b-bf20af4858b0', 'upload': '0df9156c-ef47-422c-aed4-fe25ac98d87a'}, time_total=00:13:27


{'B02': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 21, 38, 38, 532000, tzinfo=datetime.timezone.utc),info = {'dataset': 'c533b58b-9c09-4eba-9fcb-cc468aa0afc9', 'upload': '10c89ab5-d2f4-4693-8280-0b0c9eaa2aca'}, time_total = '00:13:18'),
 'B03': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 21, 51, 59, 610000, tzinfo=datetime.timezone.utc),info = {'dataset': 'c33e80a6-50d6-411c-a8b7-9397e0190e6d', 'upload': '5b52968a-ed72-400f-9c40-5d3210a206bc'}, time_total = '00:13:10'),
 'B04': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 22, 5, 10, 714000, tzinfo=datetime.timezone.utc),info = {'dataset': 'd917ff6f-5d1d-44da-a543-91509a2b7c3c', 'upload': 'eff2fa92-aac2-4d3e-bdfd-4d4703132a49'}, time_total = '00:13:20'),
 'B08': TaskStatusInfo(status='completed', time_started=datetime.datetime(2023, 3, 27, 22, 18, 31, 762000, tzinfo=datetime.timezone.utc),info = {'dataset': '0970f5a8-66cc-4348-91af-1

In [13]:
used_quota_agg = ge.get_quota(user_id)['used'] - used_quota_download
used_quota_agg

58704

Points with monthly Sentinel2 data

In [14]:
monthly_band_data_map = { name: task_status.info['dataset'] for name, task_status in status_agg.items() }

#monthly_band_data_map = {
#    'b02': 'a291c42d-ec5a-4702-954e-a76b17154752',
#    'b03': '8a15acd3-e135-4cc1-83b3-a883a52d69d7',
#    'b04': '9d3f5b09-3cb6-4dac-a673-9e5ead88a221',
#    'b08': 'd7775c4c-ab71-4f8e-8fac-ccedb154861a',
#    'ndvi': '60fb38a6-9002-4254-8554-15ac293876c7'
#    }

monthly_band_data_id_map = {name: 
    ge.api.InternalDataId(
        type="internal",
        datasetId=id) for name, id in monthly_band_data_map.items()
} 


In [15]:
points_df = gpd.read_file("group_sample_frac1_inspireId_utm32n.gpkg")
points_id = ge.upload_dataframe(points_df, "group_sample_frac1_inspireId")
points_id

1f572d66-698d-48de-92eb-e96579a1bb2c

In [16]:
points_source_operator = ge.unstable.workflow_operators.OgrSource(str(points_id))
points_source_operator.to_workflow_dict()

{'type': 'Vector',
 'operator': {'type': 'OgrSource',
  'params': {'data': {'type': 'internal',
    'datasetId': '1f572d66-698d-48de-92eb-e96579a1bb2c'},
   'attributeProjection': None,
   'attributeFilters': None}}}

In [17]:
# projected_points = ge.unstable.workflow_operators.Reprojection(points_source_operator, target_spatial_reference="EPSG:32632") # only needed if input not projected

points_with_s2_cloud_free = ge.unstable.workflow_operators.RasterVectorJoin(
    raster_sources=[ge.unstable.workflow_operators.GdalSource(x) for x in monthly_band_data_id_map.values() ],
    vector_source=points_source_operator, #projected_points,
    new_column_names=[x for x in monthly_band_data_id_map.keys()],
)

points_with_s2_cloud_free.to_workflow_dict()

workflow = ge.register_workflow(points_with_s2_cloud_free.to_workflow_dict())
workflow

fc8b6f1f-7a75-528c-a538-5ed5819ccdd9

In [18]:
start_dt = datetime(2021, 1, 1, 0, 0, 0)
end_dt = datetime(2022, 1, 1, 0, 0, 0)

start_dt, end_dt

(datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2022, 1, 1, 0, 0))

In [19]:
gp_res = await workflow.vector_stream_into_geopandas(
    ge.QueryRectangle(
        spatial_bounds=ge.BoundingBox2D(
            xmin=xmin,
            ymin=ymin,
            xmax=xmax,
            ymax=ymax,
        ),
        time_interval=ge.TimeInterval(
            start=start_dt,
            end=end_dt,
        ),
        resolution=ge.SpatialResolution(
            10.0,
            10.0,
        ),
        srs="EPSG:32632",
))

gp_res.to_file("gp_res_10_frac1_monthly_utm32n.gpkg", driver="GPKG")
gp_res

,B03,CODE,B02,ID,B04,B08,index,INSPIRE_ID,NDVI,geometry,time_start,time_end
0,NaN,311,NaN,0,NaN,NaN,0,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.033419,MULTIPOINT (428690.027 5711938.189),2021-01-01 00:00:00+00:00,2021-02-01 00:00:00+00:00
1,NaN,131,NaN,1,NaN,NaN,1,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.040039,MULTIPOINT (427819.337 5710040.545),2021-01-01 00:00:00+00:00,2021-02-01 00:00:00+00:00
2,NaN,115,NaN,2,NaN,NaN,2,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.043195,MULTIPOINT (427320.866 5710158.178),2021-01-01 00:00:00+00:00,2021-02-01 00:00:00+00:00
3,NaN,459,NaN,3,NaN,NaN,265,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.025341,MULTIPOINT (431527.388 5693772.886),2021-01-01 00:00:00+00:00,2021-02-01 00:00:00+00:00
4,NaN,459,NaN,4,NaN,NaN,266,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.016331,MULTIPOINT (431535.193 5693614.690),2021-01-01 00:00:00+00:00,2021-02-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
593059,780.0,115,596.0,49419,623.0,3531.0,732423,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.122704,MULTIPOINT (472357.075 5696612.529),2021-12-01 00:00:00+00:00,2022-01-01 00:00:00+00:00
593060,537.0,115,466.0,49420,723.0,1102.0,732424,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.093200,MULTIPOINT (472016.875 5697690.039),2021-12-01 00:00:00+00:00,2022-01-01 00:00:00+00:00
593061,530.0,115,354.0,49421,463.0,2438.0,732425,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.132107,MULTIPOINT (471981.413 5696219.338),2021-12-01 00:00:00+00:00,2022-01-01 00:00:00+00:00
593062,489.0,411,473.0,49422,609.0,1214.0,732426,https://geodaten.nrw.de/id/inspire-lu-ts/exist...,0.088917,MULTIPOINT (471704.064 5697043.769),2021-12-01 00:00:00+00:00,2022-01-01 00:00:00+00:00


In [20]:
used_quota_rvjoin = ge.get_quota(user_id)['used'] - used_quota_agg
used_quota_rvjoin

29815